In [1]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
import urllib2
import sqlite3

url = "http://db.netkeiba.com/race/201606050903/"
soup = BeautifulSoup(urllib2.urlopen(url), "html.parser")


In [3]:
dbname = 'database.db'

conn = sqlite3.connect(dbname)
c = conn.cursor()

drop_table = '''drop table race'''
c.execute(drop_table)
drop_table = '''drop table race_by_horse'''
c.execute(drop_table)
drop_table = '''drop table horse'''
c.execute(drop_table)
drop_table = '''drop table past_race'''
c.execute(drop_table)

create_table = '''create table race (race_id integer PRIMARY_KEY, name varchar(64)
                  )'''
c.execute(create_table)

create_table = '''create table race_by_horse (id integer PRIMARY KEY, horse_id int, race_id int,
                  finished int, waku_ban int, uma_ban int, name varchar(64),
                  sex int, age int, jockey_weight int, jockey_id int, jockey_name varchar(64),
                  time varchar(64), odds, popularity int, weight int)'''
c.execute(create_table)

create_table = '''create table horse (horse_id integer PRIMARY KEY, name varchar(64),
                  age int, gender varchar(32))'''
c.execute(create_table)

create_table = '''create table past_race (past_race_id integer PRIMARY KEY, horse_id int,
                  date varchar(64), place varchar(64), race_name varchar(64),
                  num_horses int, waku_ban int, uma_ban int, odds, popularity int,
                  finished int, jockey_weight int, jockey_id int, jockey_name varchar(64),
                  time varchar(64), weight int)'''
c.execute(create_table)
c.close()

In [9]:
main_table = soup.find("table", {"class":"race_table_01"})
tr_list = main_table.find_all("tr")

row_list = []
horse_list = []

for i, tr in enumerate(tr_list):
    if i == 0:
        continue
    td = tr.find("td")
    finished = td.text
    td = td.findNext("td")
    waku_ban= td.text
    td = td.findNext("td")
    uma_ban = td.text
    td = td.findNext("td")
    name = td.text
    td = td.findNext("td")
    sex_age = td.text
    sex = list(sex_age)[0]
    age = list(sex_age)[1]
    td = td.findNext("td")
    jockey_weight = td.text
    td = td.findNext("td")
    jockey_name = td.text
    td = td.findNext("td")
    time = td.text
    #diff, timeindex, passed, halftime is skipped
    td = td.findNext("td").findNext("td").findNext("td").findNext("td")
    td = td.findNext("td")
    odds = td
    td = td.findNext("td")
    popularity = td.text
    td = td.findNext("td")
    weight= td.text
    
    row = (finished, waku_ban, uma_ban, name, sex, age,
          jockey_weight, jockey_name, time, odds, popularity, weight)
    row_list.append(row)
print len(row_list)

AttributeError: 'list' object has no attribute 'size'

In [ ]:
dbname = 'database.db'

conn = sqlite3.connect(dbname)
c = conn.cursor()

insert_sql = 'insert into users (id, name, age, gender) values (?,?,?,?)'
users = [
    (2, 'Shota', 54, 'male'),
    (3, 'Nana', 40, 'female'),
    (4, 'Tooru', 78, 'male'),
    (5, 'Saki', 31, 'female')
]
c.executemany(insert_sql, users)
conn.commit()
conn.close()

In [44]:
# -*- coding: utf-8

import sqlite3

dbname = 'database.db'

conn = sqlite3.connect(dbname)
c = conn.cursor()

# executeメソッドでSQL文を実行する
create_table = '''create table users (id int, name varchar(64),
                  age int, gender varchar(32))'''
c.execute(create_table)

# SQL文に値をセットする場合は，Pythonのformatメソッドなどは使わずに，
# セットしたい場所に?を記述し，executeメソッドの第2引数に?に当てはめる値を
# タプルで渡す．
sql = 'insert into users (id, name, age, gender) values (?,?,?,?)'
user = (1, 'Taro', 20, 'male')
c.execute(sql, user)

# 一度に複数のSQL文を実行したいときは，タプルのリストを作成した上で
# executemanyメソッドを実行する
insert_sql = 'insert into users (id, name, age, gender) values (?,?,?,?)'
users = [
    (2, 'Shota', 54, 'male'),
    (3, 'Nana', 40, 'female'),
    (4, 'Tooru', 78, 'male'),
    (5, 'Saki', 31, 'female')
]
c.executemany(insert_sql, users)
conn.commit()

select_sql = 'select * from users'
for row in c.execute(select_sql):
    print(row)

conn.close()

(1, u'Taro', 20, u'male')
(2, u'Shota', 54, u'male')
(3, u'Nana', 40, u'female')
(4, u'Tooru', 78, u'male')
(5, u'Saki', 31, u'female')
